Create bash scripts

In [1]:
import os
from pathlib import Path
import numpy as np
from copy import deepcopy

from light_by_light.vacem_ini import W_to_E0, create_ini_file
from light_by_light.utils import write_yaml, read_yaml

/home/wi73yus/.conda/envs/lbl/lib/python3.9/site-packages/pyfftw/interfaces/__init__.py:268: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  has_scipy_fft = _LooseVersion(scipy.__version__) >= _LooseVersion('1.4.0')


In [2]:
template = '''#!/bin/bash
#SBATCH --job-name={jobname}
#SBATCH --partition=gpu
#SBATCH --cpus-per-task={n_cpus}
#SBATCH --time=10:00:00
#SBATCH --mem=150GB
#SBATCH --output=/work/wi73yus/bash_output/{jobname}.%j.log
#SBATCH --error=/work/wi73yus/bash_output/{jobname}.%j.err

source ~/.bashrc  #  not automatically read by slurm
conda activate lbl  # activate here the anaconda environment you want to use

python {script_path} {script_kwargs}
'''

home_path = '/home/wi73yus/light-by-light'
work_path = '/work/wi73yus/light-by-light-experiments'

### Bash single simulation

#### ECS

In [4]:
# Laser pulse parameters
theta = 160
beta = 90

# 1st laser
laser_params = []
laser = {
    'tau': 25e-15,
    'lam': 800e-9,
    'w0x': 5.5*800e-9,
    'w0y': 1*800e-9,
    'W': 25,
    'theta': 0,
    'phi': 0,
    'beta': 0,
    'solution': 'gaussian_elliptical_paraxial'
}
laser['E0'] = float(W_to_E0(laser))
laser_params.append(laser)

# 2nd laser
laser = {
    'tau': 25e-15,
    'lam': 800e-9,
    'w0': 1*800e-9,
    'W': 25,
    'theta': theta,
    'phi': 0,
    'beta': beta,
    'solution': 'gaussian_paraxial'
}
laser['E0'] = float(W_to_E0(laser))
laser_params.append(laser)

laser_params = {f'laser_{i}': value for i,value in enumerate(laser_params)}

# Simulation box parameters
box_factors = {
    'long': 7,
    'trans': 15,
    't': 2
}

resolutions = {
    'spatial': 2,
    't': 1
}
simbox_params = {'box_factors': box_factors,
                 'resolutions': resolutions}

# Other parameters
mode = 'simulation'
geometry = 'z'
low_memory_mode = False
n_threads = 24
pol_idx = 0

discernible_spectral = False
sphmap_params = {
    'order': 1,
    'angular_resolution': 0.5/180*np.pi
}

# Save yaml file
data = {
    '_mode': mode,
    'lasers': laser_params,
    'simbox_params': simbox_params,
    'geometry': geometry,
    'low_memory_mode': low_memory_mode,
    'n_threads': n_threads,
    'pol_idx': pol_idx,
    'discernible_spectral': discernible_spectral,
    'sphmap_params': sphmap_params,
}

folder = f'ECS_W'
jobname = f'theta_{theta}_w0x_5.5_w0y_1'
save_path = f'{work_path}/runs/{mode}/{folder}/{jobname}/'
Path(os.path.dirname(save_path)).mkdir(parents=True, exist_ok=True)

default_yaml = f'{save_path}default_params.yml'
write_yaml(default_yaml, data)

# Create bash script
script_path = f'{home_path}/cluster/simulation_cluster.py'
script_kwargs = f'--save_path={save_path}'

bash = template.format(jobname=jobname,
                       n_cpus=n_threads,
                       script_path=script_path,
                       script_kwargs=script_kwargs)

bash_path = f'{save_path}job.slurm'
with open(bash_path, 'w') as f:
    f.write(bash)

In [5]:
default_name = 'default_params.yml'

default_yaml = f'{save_path}{default_name}'
yaml_data = read_yaml(default_yaml)
params = {}
lasers = yaml_data['lasers']
params['laser_params'] = [lasers[f'laser_{i}'] for i in range(len(lasers))]
params['save_path'] = save_path
keys = ['simbox_params', 'geometry', 'low_memory_mode']
for key in keys:
    params[key] = yaml_data[key]

create_ini_file(**params)

In [6]:
os.system(f'sbatch {bash_path}')

Submitted batch job 924401


0

#### Harmonics

In [41]:
# Laser pulse parameters
theta = 180
beta = 90

Wtotal = 25
H = 1 + 1/2**2.5
W = [Wtotal/(n**2.5*H) for n in [1,2]]
# print(W)
# print(np.sum(W))

W = [25, 25]

# 1st laser
lam0 = 800e-9
k0 = 2*np.pi/lam0
order = 0
gauss_type = 'gaussian_spectral'

w0_harm = 1
w0_usual = 1
phi0_harm = [0, 0]

laser_params = []
for i in range(1,len(W)+1):
    laser = {
        'tau': 20e-15,
        'lam': lam0/i,
        'w0': w0_harm*800e-9/i,
        'W': W[i-1],
        'theta': 0,
        'phi': 0,
        'beta': 0,
        'solution': gauss_type,
        'order': order,
        'phi0': phi0_harm[i-1],
    }
    laser['E0'] = float(W_to_E0(laser))
    laser_params.append(laser)

# 2nd laser
laser = {
    'tau': 20e-15,
    'lam': lam0,
    'w0': w0_usual*800e-9,
    'W': 25,
    'theta': theta,
    'phi': 0,
    'beta': beta,
    'solution': gauss_type,
    'order': order,
    'phi0': 0
}
laser['E0'] = float(W_to_E0(laser))
laser_params.append(laser)

laser_params = {f'laser_{i}': value for i,value in enumerate(laser_params)}

# Simulation box parameters
box_factors = {
    'long': 8,
    'trans': 18,
    't': 2
}

resolutions = {
    'spatial': 3,
    't': 2
}
simbox_params = {'box_factors': box_factors,
                 'resolutions': resolutions}

# Other parameters
mode = 'simulation'
geometry = 'z'
low_memory_mode = False
n_threads = 24
pol_idx = 0

discernible_spectral = True
sphmap_params = {
    'order': 1,
    'angular_resolution': 0.5/180*np.pi
}

sph_limits = {
    'k': [1.5*k0, 2.5*k0],
    'theta': [np.pi/2, np.pi]
}

# Save yaml file
data = {
    '_mode': mode,
    'lasers': laser_params,
    'simbox_params': simbox_params,
    'geometry': geometry,
    'low_memory_mode': low_memory_mode,
    'n_threads': n_threads,
    'pol_idx': pol_idx,
    'discernible_spectral': discernible_spectral,
    'sphmap_params': sphmap_params,
    'sph_limits': sph_limits
}

folder = f'harmonics_W'
if type(w0_harm) == int:
    w0_str = str(w0_harm)
else:
    w0_str = '_'.join(str(w0) for w0 in w0_harm)
jobname = f'21-1omega_theta_{theta}_w0harm_{w0_str}_figure'
save_path = f'{work_path}/runs/{mode}/{folder}/{jobname}/'
Path(os.path.dirname(save_path)).mkdir(parents=True, exist_ok=True)

default_yaml = f'{save_path}default_params.yml'
write_yaml(default_yaml, data)

# Create bash script
script_path = f'{home_path}/cluster/simulation_cluster.py'
script_kwargs = f'--save_path={save_path}'

bash = template.format(jobname=jobname,
                       n_cpus=n_threads,
                       script_path=script_path,
                       script_kwargs=script_kwargs)

bash_path = f'{save_path}job.slurm'
with open(bash_path, 'w') as f:
    f.write(bash)

In [42]:
default_name = 'default_params.yml'

default_yaml = f'{save_path}{default_name}'
yaml_data = read_yaml(default_yaml)
params = {}
lasers = yaml_data['lasers']
params['laser_params'] = [lasers[f'laser_{i}'] for i in range(len(lasers))]
params['save_path'] = save_path
keys = ['simbox_params', 'geometry', 'low_memory_mode']
for key in keys:
    params[key] = yaml_data[key]

create_ini_file(**params)

In [43]:
os.system(f'sbatch {bash_path}')

Submitted batch job 923820


0

#### 3 pulses collision

In [10]:
# Laser pulse parameters
theta = 180
beta = 90

# 1st laser
laser_params = []
laser = {
    'tau': 25e-15,
    'lam': 800e-9,
    'w0': 3*800e-9,
    'W': 50,
    'theta': 0,
    'phi': 0,
    'beta': 0,
    'solution': 'gaussian_paraxial'
}
laser['E0'] = float(W_to_E0(laser))
laser_params.append(laser)

# # 2nd laser
# laser = {
#     'tau': 25e-15,
#     'lam': 800e-9,
#     'w0': 3*800e-9,
#     'W': 25,
#     'theta': 0,
#     'phi': 0,
#     'beta': 0,
#     'solution': 'gaussian_paraxial'
# }
# laser['E0'] = float(W_to_E0(laser))
# laser_params.append(laser)

# 3rd laser
laser = {
    'tau': 25e-15,
    'lam': 800e-9,
    'w0': 1*800e-9,
    'W': 25,
    'theta': theta,
    'phi': 0,
    'beta': beta,
    'solution': 'gaussian_paraxial'
}
laser['E0'] = float(W_to_E0(laser))
laser_params.append(laser)

laser_params = {f'laser_{i}': value for i,value in enumerate(laser_params)}

# Simulation box parameters
box_factors = {
    'long': 7,
    'trans': 15,
    't': 2
}

resolutions = {
    'spatial': 2,
    't': 1
}
simbox_params = {'box_factors': box_factors,
                 'resolutions': resolutions,
                 'pick_largest_size': True}

# Other parameters
mode = 'simulation'
geometry = 'xz'
low_memory_mode = False
n_threads = 32
pol_idx = 0

discernible_spectral = False
sphmap_params = {
    'order': 1,
    'angular_resolution': 0.5/180*np.pi
}

# Save yaml file
data = {
    '_mode': mode,
    'lasers': laser_params,
    'simbox_params': simbox_params,
    'geometry': geometry,
    'low_memory_mode': low_memory_mode,
    'n_threads': n_threads,
    'pol_idx': pol_idx,
    'discernible_spectral': discernible_spectral,
    'sphmap_params': sphmap_params,
}

folder = f'3_pulses'
jobname = f'w0_3_counter_W_50'
save_path = f'{work_path}/runs/{mode}/{folder}/{jobname}/'
Path(os.path.dirname(save_path)).mkdir(parents=True, exist_ok=True)

default_yaml = f'{save_path}default_params.yml'
write_yaml(default_yaml, data)

# Create bash script
script_path = f'{home_path}/cluster/simulation_cluster.py'
script_kwargs = f'--save_path={save_path}'

bash = template.format(jobname=jobname,
                       n_cpus=n_threads,
                       script_path=script_path,
                       script_kwargs=script_kwargs)

bash_path = f'{save_path}job.slurm'
with open(bash_path, 'w') as f:
    f.write(bash)

In [11]:
default_name = 'default_params.yml'

default_yaml = f'{save_path}{default_name}'
yaml_data = read_yaml(default_yaml)
params = {}
lasers = yaml_data['lasers']
params['laser_params'] = [lasers[f'laser_{i}'] for i in range(len(lasers))]
params['save_path'] = save_path
keys = ['simbox_params', 'geometry', 'low_memory_mode']
for key in keys:
    params[key] = yaml_data[key]

create_ini_file(**params)

In [12]:
os.system(f'sbatch {bash_path}')

Submitted batch job 1055857


0

### Bash script for 1d grid scan

#### CCS

In [20]:
# Laser pulse parameters
theta = 180
beta = 90

# tau = float(25e-15 * 2 * np.sqrt(np.log(2)))
# 1st laser
laser_params = []
laser = {
    'tau': 25e-15,
    'lam': 800e-9,
    'w0': 3*800e-9,
    'W': 25,
    'theta': 0,
    'phi': 0,
    'beta': 0,
    'solution': 'gaussian_paraxial'
}
laser['E0'] = float(W_to_E0(laser))
laser_params.append(laser)

# 2nd laser
laser = {
    'tau': 25e-15,
    'lam': 800e-9,
    'w0': 1*800e-9,
    'W': 25,
    'theta': theta,
    'phi': 0,
    'beta': beta,
    'solution': 'gaussian_paraxial'
}
laser['E0'] = float(W_to_E0(laser))
laser_params.append(laser)

laser_params = {f'laser_{i}': value for i,value in enumerate(laser_params)}

# Simulation box parameters
box_factors = {
    'long': 7,
    'trans': 15,
    't': 2
}

resolutions = {
    'spatial': 2,
    't': 1
}
simbox_params = {'box_factors': box_factors,
                 'resolutions': resolutions}

# Other parameters
mode = 'gridscan'
geometry = 'z'
low_memory_mode = False
n_threads = 24
pol_idx = 0

# Variable parameters 
laser_key = 'laser_0'
param_key = 'W'
grid = [1,50,10,1]
vary_params = {
    'lasers': {laser_key: {param_key: grid}}
}

# Save yaml file
data = {
    '_mode': mode,
    'lasers': laser_params,
    'simbox_params': simbox_params,
    'geometry': geometry,
    'low_memory_mode': low_memory_mode,
    'n_threads': n_threads,
    'pol_idx': pol_idx,
    'do_energy_budget': False,
}

folder = f'CCS_W'
jobname = f'{param_key}_theta_{theta}_{geometry}'
save_path = f'{work_path}/runs/{mode}/{folder}/{jobname}/'
Path(os.path.dirname(save_path)).mkdir(parents=True, exist_ok=True)

default_yaml = f'{save_path}default_params.yml'
write_yaml(default_yaml, data)

vary_yaml = f'{save_path}vary_params.yml'
write_yaml(vary_yaml, vary_params)

# Create bash script
script_path = f'{home_path}/cluster/gridscan_cluster.py'
script_kwargs = f'--save_path={save_path}'

bash = template.format(jobname=jobname,
                       n_cpus=n_threads,
                       script_path=script_path,
                       script_kwargs=script_kwargs)

bash_path = f'{save_path}job.slurm'
with open(bash_path, 'w') as f:
    f.write(bash)

In [21]:
os.system(f'sbatch {bash_path}')

Submitted batch job 1055918


0

#### Harmonics

In [55]:
# Laser pulse parameters
theta = 180
beta = 90

Wtotal = 25
H = 1 + 1/2**2.5
W = [Wtotal/(n**2.5*H) for n in [1,2]]
# print(W)
# print(np.sum(W))

n = 2
W = [50/n for i in range(n)]

# 1st laser
lam0 = 800e-9
k0 = 2*np.pi/lam0
order = 0
gauss_type = 'gaussian_spectral'

w0_harm = 1
w0_usual = 1
phi0_harm = [0 for i in range(n)]

laser_params = []
for i in range(1,n+1):
    laser = {
        'tau': 20e-15,
        'lam': lam0/i,
        'w0': w0_harm*800e-9/i,
        'W': W[i-1],
        'theta': 0,
        'phi': 0,
        'beta': 0,
        'solution': gauss_type,
        'order': order,
        'phi0': phi0_harm[i-1],
    }
    laser['E0'] = float(W_to_E0(laser))
    laser_params.append(laser)

# 2nd laser
laser = {
    'tau': 20e-15,
    'lam': lam0,
    'w0': w0_usual*800e-9,
    'W': 25,
    'theta': theta,
    'phi': 0,
    'beta': beta,
    'solution': gauss_type,
    'order': order,
    'phi0': 0
}
laser['E0'] = float(W_to_E0(laser))
laser_params.append(laser)

laser_params = {f'laser_{i}': value for i,value in enumerate(laser_params)}

# Simulation box parameters
box_factors = {
    'long': 7,
    'trans': 15,
    't': 2
}

resolutions = {
    'spatial': 3,
    't': 1
}
simbox_params = {'box_factors': box_factors,
                 'resolutions': resolutions}

# Other parameters
mode = 'gridscan'
geometry = 'z'
low_memory_mode = False
n_threads = 24
pol_idx = 0

# Variable parameters 
laser_key = 'laser_1'
param_key = 'phi0'
grid = [0,330,12]
vary_params = {
    'lasers': {laser_key: {param_key: grid}}
}

discernible_spectral = True
sphmap_params = {
    'order': 1,
    'angular_resolution': 0.5/180*np.pi
}

sph_limits = {
    'k': [1.5*k0, 2.5*k0],
    'theta': [np.pi/2, np.pi]
}

# Save yaml file
data = {
    '_mode': mode,
    'lasers': laser_params,
    'simbox_params': simbox_params,
    'geometry': geometry,
    'low_memory_mode': low_memory_mode,
    'n_threads': n_threads,
    'pol_idx': pol_idx,
    'discernible_spectral': discernible_spectral,
    'sphmap_params': sphmap_params,
    'sph_limits': sph_limits
}

folder = f'harmonics_W'
if type(w0_harm) == int:
    w0_str = str(w0_harm)
else:
    w0_str = '_'.join(str(w0) for w0 in w0_harm)
jobname = f'21-1omega_theta_{theta}_w0harm_{w0_str}_phase'
save_path = f'{work_path}/runs/{mode}/{folder}/{jobname}/'
Path(os.path.dirname(save_path)).mkdir(parents=True, exist_ok=True)

default_yaml = f'{save_path}default_params.yml'
write_yaml(default_yaml, data)

vary_yaml = f'{save_path}vary_params.yml'
write_yaml(vary_yaml, vary_params)

# Create bash script
script_path = f'{home_path}/cluster/gridscan_cluster.py'
script_kwargs = f'--save_path={save_path}'

bash = template.format(jobname=jobname,
                       n_cpus=n_threads,
                       script_path=script_path,
                       script_kwargs=script_kwargs)

bash_path = f'{save_path}job.slurm'
with open(bash_path, 'w') as f:
    f.write(bash)

In [56]:
os.system(f'sbatch {bash_path}')

Submitted batch job 923887


0

### Bash script for optuna optimization

In [3]:
# Laser pulse parameters
theta = 180
beta = 90

# 1st laser
laser_params = []
laser = {
    'tau': 25e-15,
    'lam': 800e-9,
    'w0': 4*800e-9,
    'W': 25,
    'theta': 0,
    'phi': 0,
    'beta': 0,
    'solution': 'gaussian_paraxial'
}
laser['E0'] = float(W_to_E0(laser))
laser_params.append(laser)

# 2nd laser
laser = {
    'tau': 25e-15,
    'lam': 800e-9,
    'w0': 1*800e-9,
    'W': 25,
    'theta': theta,
    'phi': 0,
    'beta': beta,
    'solution': 'gaussian_paraxial'
}
laser['E0'] = float(W_to_E0(laser))
laser_params.append(laser)

laser_params = {f'laser_{i}': value for i,value in enumerate(laser_params)}

# Simulation box parameters
box_factors = {
    'long': 8,
    'trans': 20,
    't': 2
}

resolutions = {
    'spatial': 2,
    't': 1
}
simbox_params = {'box_factors': box_factors,
                 'resolutions': resolutions}

# Other parameters
mode = 'optuna'
geometry = 'xz'
low_memory_mode = False
n_threads = 24
pol_idx = 0

# Optuna params
obj_param = 'N_disc_num'
n_trials = 5

data = {
    '_mode': mode,
    'lasers': laser_params,
    'simbox_params': simbox_params,
    'geometry': geometry,
    'low_memory_mode': low_memory_mode,
    'n_threads': n_threads,
    'pol_idx': pol_idx,
    'obj_param': obj_param,
    'n_trials': n_trials
}

# Variable parameters 
laser_key = 'laser_1'
param_key = 'beta'
grid = [0,180,1,'uniform']
vary_params = {laser_key:
    {param_key: grid}
}

# Save yaml files
folder = f'CCS'
jobname = f'{param_key}_theta_{theta}'
save_path = f'{work_path}/runs/{mode}/{folder}/{jobname}/'
Path(os.path.dirname(save_path)).mkdir(parents=True, exist_ok=True)

default_yaml = f'{save_path}default_params.yml'
write_yaml(default_yaml, data)

vary_yaml = f'{save_path}optuna_params.yml'
write_yaml(vary_yaml, vary_params)

script_path = f'{home_path}/cluster/optuna_cluster.py'
script_kwargs = f'--save_path={save_path}'

bash = template.format(jobname=f'opt_{jobname}',
                       n_cpus=n_threads,
                       script_path=script_path,
                       script_kwargs=script_kwargs)

bash_path = f'{save_path}job.slurm'
with open(bash_path, 'w') as f:
    f.write(bash)

In [4]:
os.system(f'sbatch {bash_path}')

Submitted batch job 600622


0

### Bash for numerical consistency check

In [28]:
template_res = '''#!/bin/bash
#SBATCH --job-name={jobname}
#SBATCH --partition=hij-gpu
#SBATCH --cpus-per-task={n_cpus}
#SBATCH --time=8:00:00
#SBATCH --mem=200GB
#SBATCH --output=/work/wi73yus/bash_output/{jobname}.%A_%a.log
#SBATCH --error=/work/wi73yus/bash_output/{jobname}.%A_%a.err
#SBATCH --array=0-4

arr=$SLURM_ARRAY_TASK_ID
echo $arr

source ~/.bashrc  #  not automatically read by slurm
conda activate lbl  # activate here the anaconda environment you want to use

python {script_path} {script_kwargs}
'''

#### ECS

In [39]:
# We need to test the same scattering configuration with a couple of 
# better numerical resolutions
# 1) increase time resolution,
# 2) increase spatial resolution
# 3) increase box size in time
# 4) increase box size in space

# Laser pulse parameters
theta = 180
beta = 90

# 1st laser
laser_params = []
laser = {
    'tau': 25e-15,
    'lam': 800e-9,
    'w0x': 4*800e-9,
    'w0y': 1*800e-9,
    'W': 25,
    'theta': 0,
    'phi': 0,
    'beta': 0,
    'solution': 'gaussian_elliptical_paraxial'
}
laser['E0'] = float(W_to_E0(laser))
laser_params.append(laser)

# 2nd laser
laser = {
    'tau': 25e-15,
    'lam': 800e-9,
    'w0': 1*800e-9,
    'W': 25,
    'theta': theta,
    'phi': 0,
    'beta': beta,
    'solution': 'gaussian_paraxial'
}
laser['E0'] = float(W_to_E0(laser))
laser_params.append(laser)

laser_params = {f'laser_{i}': value for i,value in enumerate(laser_params)}

# Simulation box parameters
box_factors = {
    'long': 8,
    'trans': 20,
    't': 2
}

resolutions = {
    'spatial': 2,
    't': 1
}
simbox_params = {'box_factors': box_factors,
                 'resolutions': resolutions}

# Other parameters
mode = 'gridscan'
geometry = 'xz'
low_memory_mode = False
n_threads = 12
pol_idx = 0

data = {
    'lasers': laser_params,
    'simbox_params': simbox_params,
    'geometry': geometry,
    'low_memory_mode': low_memory_mode,
    'n_threads': n_threads,
    'pol_idx': pol_idx,
}

simbox_cases = {f'case_{i}': deepcopy(simbox_params) for i in range(0,5)}

simbox_cases['case_1']['resolutions']['t'] = 2
simbox_cases['case_2']['resolutions']['spatial'] = 3
simbox_cases['case_3']['box_factors']['t'] = 4
simbox_cases['case_4']['box_factors']['long'] = 10
simbox_cases['case_4']['box_factors']['trans'] = 25

folder = f'CCS'
jobname = f'theta_{theta}'
save_path = f'{work_path}/runs/resolution_check/{folder}/{jobname}/'

for key in simbox_cases.keys():
    data['simbox_params'] = simbox_cases[key]
    default_yaml = f'{save_path}{key}/default_params.yml'
    Path(os.path.dirname(default_yaml)).mkdir(parents=True, exist_ok=True)
    write_yaml(default_yaml, data)

In [41]:
script_path = f'{home_path}/cluster/simulation_cluster.py'
save_path_arr = save_path + 'case_$arr/'
script_kwargs = f'--save_path={save_path_arr}'

bash = template_res.format(jobname=f'numcheck_{jobname}',
                           n_cpus=n_threads,
                           script_path=script_path,
                           script_kwargs=script_kwargs)

bash_path = f'{save_path}job.slurm'
with open(bash_path, 'w') as f:
    f.write(bash)

In [42]:
os.system(f'sbatch {bash_path}')

Submitted batch job 600314


0

#### Harmonics

In [29]:
# Laser pulse parameters
theta = 180
beta = 90

Wtotal = 25
H = 1 + 1/2**2.5
W = [Wtotal/(n**2.5*H) for n in [1,2]]
print(W)
print(np.sum(W))

# 1st laser
w0_harm = 1
w0_usual = 1
# 1st laser
laser_params = []
for i in range(1,3):
    laser = {
        'tau': 20e-15,
        'lam': 800e-9/i,
        'w0': w0_harm*800e-9/i,
        'W': W[i-1],
        'theta': 0,
        'phi': 0,
        'beta': 0,
        'solution': 'gaussian_paraxial'
    }
    laser['E0'] = float(W_to_E0(laser))
    laser_params.append(laser)

# 2nd laser
laser = {
    'tau': 20e-15,
    'lam': 800e-9,
    'w0': w0_usual*800e-9,
    'W': 25,
    'theta': theta,
    'phi': 0,
    'beta': beta,
    'solution': 'gaussian_paraxial'
}
laser['E0'] = float(W_to_E0(laser))
laser_params.append(laser)

laser_params = {f'laser_{i}': value for i,value in enumerate(laser_params)}

# Simulation box parameters
box_factors = {
    'long': 7,
    'trans': 15,
    't': 2
}

resolutions = {
    'spatial': 2,
    't': 1
}
simbox_params = {'box_factors': box_factors,
                 'resolutions': resolutions}

# Other parameters
mode = 'resolution_check'
geometry = 'z'
low_memory_mode = False
n_threads = 24
pol_idx = 0

discernible_spectral = True
sphmap_params = {
    'order': 1,
    'angular_resolution': 0.5/180*np.pi
}

# Save yaml file
data = {
    '_mode': mode,
    'lasers': laser_params,
    'simbox_params': simbox_params,
    'geometry': geometry,
    'low_memory_mode': low_memory_mode,
    'n_threads': n_threads,
    'pol_idx': pol_idx,
    'discernible_spectral': discernible_spectral,
    'sphmap_params': sphmap_params
}

simbox_cases = {f'case_{i}': deepcopy(simbox_params) for i in range(0,5)}

simbox_cases['case_1']['resolutions']['t'] = 2
simbox_cases['case_2']['resolutions']['spatial'] = 4
simbox_cases['case_3']['box_factors']['t'] = 4
simbox_cases['case_4']['box_factors']['long'] = 10
simbox_cases['case_4']['box_factors']['trans'] = 25

folder = f'harmonics'
jobname = f'21-1omega_theta_{theta}_beta_{beta}_w0harm_{w0_harm}_w0usual_{w0_usual}'
save_path = f'{work_path}/runs/{mode}/{folder}/{jobname}/'

for key in simbox_cases.keys():
    data['simbox_params'] = simbox_cases[key]
    default_yaml = f'{save_path}{key}/default_params.yml'
    Path(os.path.dirname(default_yaml)).mkdir(parents=True, exist_ok=True)
    write_yaml(default_yaml, data)

default_yaml = f'{save_path}default_params.yml'
write_yaml(default_yaml, data)

# Create bash script
script_path = f'{home_path}/cluster/simulation_cluster.py'
save_path_arr = save_path + 'case_$arr/'
script_kwargs = f'--save_path={save_path_arr}'

bash = template_res.format(jobname=f'numcheck_{jobname}',
                           n_cpus=n_threads,
                           script_path=script_path,
                           script_kwargs=script_kwargs)

bash_path = f'{save_path}job.slurm'
with open(bash_path, 'w') as f:
    f.write(bash)

[21.24447237944163, 3.7555276205583703]
25.0


In [30]:
os.system(f'sbatch {bash_path}')

Submitted batch job 721425


0

### Bash for 2d grid scan where one dimension is performed in parallel

In [9]:
template_2d_gridscan = '''#!/bin/bash
#SBATCH --job-name={jobname}
#SBATCH --partition=hij-gpu
#SBATCH --cpus-per-task={n_cpus}
#SBATCH --time=24:00:00
#SBATCH --mem=250GB
#SBATCH --output=/work/wi73yus/bash_output/{jobname}.%A_%a.log
#SBATCH --error=/work/wi73yus/bash_output/{jobname}.%A_%a.err
#SBATCH --array={array_size}

source ~/.bashrc  #  not automatically read by slurm
conda activate lbl  # activate here the anaconda environment you want to use

echo $SLURM_ARRAY_TASK_ID

python {script_path} {script_kwargs}
'''

#### ECS

In [10]:
# Laser pulse parameters
theta = 180
beta = 90

# 1st laser
laser_params = []
laser = {
    'tau': 25e-15,
    'lam': 800e-9,
    'w0x': 1*800e-9,
    'w0y': 1*800e-9,
    'W': 25,
    'theta': 0,
    'phi': 0,
    'beta': 0,
    'solution': 'gaussian_elliptical_paraxial'
}
laser['E0'] = float(W_to_E0(laser))
laser_params.append(laser)

# 2nd laser
laser = {
    'tau': 25e-15,
    'lam': 800e-9,
    'w0': 1*800e-9,
    'W': 25,
    'theta': theta,
    'phi': 0,
    'beta': beta,
    'solution': 'gaussian_paraxial'
}
laser['E0'] = float(W_to_E0(laser))
laser_params.append(laser)

laser_params = {f'laser_{i}': value for i,value in enumerate(laser_params)}

# Simulation box parameters
box_factors = {
    'long': 7,
    'trans': 15,
    't': 2
}

resolutions = {
    'spatial': 2,
    't': 1
}
simbox_params = {'box_factors': box_factors,
                 'resolutions': resolutions}

# Other parameters
mode = 'gridscan'
geometry = 'z'
low_memory_mode = False
n_threads = 18
pol_idx = 0

# Variable parameters 
laser_key = 'laser_0'
param_key = 'w0x'
grid = [1,9,9,800e-9]
vary_params = {
    'lasers': {laser_key: {param_key: grid}}
}

# Additional variable parameter
laser_key_add = 'laser_0'
param_key_add = 'w0y'
grid = [1,9,9,800e-9]
param_grid = np.linspace(grid[0], grid[1], grid[2], endpoint=True)

for idx, param_value in enumerate(param_grid):
    laser_params[laser_key_add][param_key_add] = float(param_value*grid[-1])
    laser_params[laser_key_add]['E0'] = float(W_to_E0(laser_params[laser_key_add]))

    # Save yaml file
    data = {
        '_mode': mode,
        'lasers': laser_params,
        'simbox_params': simbox_params,
        'geometry': geometry,
        'low_memory_mode': low_memory_mode,
        'n_threads': n_threads,
        'pol_idx': pol_idx
    }

    folder = f'ECS_W'
    jobname = f'{param_key}_theta_{theta}_{geometry}'
    save_path = f'{work_path}/runs/{mode}/{folder}/{jobname}/'
    save_path_local = f'{save_path}/{idx}/'
    Path(os.path.dirname(save_path_local)).mkdir(parents=True, exist_ok=True)

    default_yaml = f'{save_path_local}default_params.yml'
    write_yaml(default_yaml, data)

    vary_yaml = f'{save_path_local}vary_params.yml'
    write_yaml(vary_yaml, vary_params)

# Create bash script
script_path = f'{home_path}/cluster/gridscan_cluster.py'
script_kwargs = f'--save_path={save_path}$SLURM_ARRAY_TASK_ID/'
array_size = f'0-{len(param_grid)-1}'

bash = template_2d_gridscan.format(jobname=jobname,
                       n_cpus=n_threads,
                       array_size=array_size,
                       script_path=script_path,
                       script_kwargs=script_kwargs)

bash_path = f'{save_path}job.slurm'
with open(bash_path, 'w') as f:
    f.write(bash)

In [11]:
os.system(f'sbatch {bash_path}')

Submitted batch job 924422


0

#### Harmonics

In [6]:
# Laser pulse parameters
theta = 180
beta = 90

Wtotal = 25
H = 1 + 1/2**2.5
W = [Wtotal/(n**2.5*H) for n in [1,2]]
# print(W)
# print(np.sum(W))

n = 3
W = [50/n for i in range(n)]

# 1st laser
lam0 = 800e-9
k0 = 2*np.pi/lam0
order = 0
gauss_type = 'gaussian_spectral'

w0_harm = 1
w0_usual = 1
phi0_harm = [0 for i in range(n)]

laser_params = []
for i in range(1,4):
    laser = {
        'tau': 20e-15,
        'lam': lam0/i,
        'w0': w0_harm*800e-9/i,
        'W': W[i-1],
        'theta': 0,
        'phi': 0,
        'beta': 0,
        'solution': gauss_type,
        'order': order,
        'phi0': phi0_harm[i-1],
    }
    laser['E0'] = float(W_to_E0(laser))
    laser_params.append(laser)

# 2nd laser
laser = {
    'tau': 20e-15,
    'lam': lam0,
    'w0': w0_usual*800e-9,
    'W': 25,
    'theta': theta,
    'phi': 0,
    'beta': beta,
    'solution': gauss_type,
    'order': order,
    'phi0': 0
}
laser['E0'] = float(W_to_E0(laser))
laser_params.append(laser)

laser_params = {f'laser_{i}': value for i,value in enumerate(laser_params)}

# Simulation box parameters
box_factors = {
    'long': 7,
    'trans': 15,
    't': 2
}

resolutions = {
    'spatial': 2,
    't': 1
}
simbox_params = {'box_factors': box_factors,
                 'resolutions': resolutions}

# Other parameters
mode = 'gridscan'
geometry = 'z'
low_memory_mode = False
n_threads = 24
pol_idx = 0

# Variable parameters 
laser_key = 'laser_1'
param_key = 'phi0'
grid = [0,330,12]
vary_params = {
    'lasers': {laser_key: {param_key: grid}}
}

# Additional variable parameter
laser_key_add = 'laser_2'
param_key_add = 'phi0'
grid = [0,330,12]
param_grid = np.linspace(grid[0], grid[1], grid[2], endpoint=True)

for idx, param_value in enumerate(param_grid):
    laser_params[laser_key_add][param_key_add] = float(param_value)
    laser_params[laser_key_add]['E0'] = float(W_to_E0(laser_params[laser_key_add]))

    # Save yaml file
    data = {
        '_mode': mode,
        'lasers': laser_params,
        'simbox_params': simbox_params,
        'geometry': geometry,
        'low_memory_mode': low_memory_mode,
        'n_threads': n_threads,
        'pol_idx': pol_idx
    }

    folder = f'harmonics_W'
    jobname = f'{param_key}_{laser_key}_theta_{theta}_{geometry}'
    save_path = f'{work_path}/runs/{mode}/{folder}/{jobname}/'
    save_path_local = f'{save_path}/{idx}/'
    Path(os.path.dirname(save_path_local)).mkdir(parents=True, exist_ok=True)

    default_yaml = f'{save_path_local}default_params.yml'
    write_yaml(default_yaml, data)

    vary_yaml = f'{save_path_local}vary_params.yml'
    write_yaml(vary_yaml, vary_params)

# Create bash script
script_path = f'{home_path}/cluster/gridscan_cluster.py'
script_kwargs = f'--save_path={save_path}$SLURM_ARRAY_TASK_ID/'
array_size = f'0-{len(param_grid)-1}'

bash = template_2d_gridscan.format(jobname=jobname,
                       n_cpus=n_threads,
                       array_size=array_size,
                       script_path=script_path,
                       script_kwargs=script_kwargs)

bash_path = f'{save_path}job.slurm'
with open(bash_path, 'w') as f:
    f.write(bash)

In [7]:
os.system(f'sbatch {bash_path}')

Submitted batch job 924025


0

### Bash for optuna optimization from several jobs

In [5]:
template_optuna = '''#!/bin/bash
#SBATCH --job-name={jobname}
#SBATCH --partition=gpu
#SBATCH --cpus-per-task={n_cpus}
#SBATCH --time=24:00:00
#SBATCH --mem=200GB
#SBATCH --output=/work/wi73yus/bash_output/{jobname}.%A_%a.log
#SBATCH --error=/work/wi73yus/bash_output/{jobname}.%A_%a.err
#SBATCH --array=0-1

source ~/.bashrc  #  not automatically read by slurm
conda activate lbl  # activate here the anaconda environment you want to use

echo $SLURM_ARRAY_TASK_ID

if [ $SLURM_ARRAY_TASK_ID -gt 0 ]
then
    sleep 300
fi

python {script_path} {script_kwargs}
'''

#### CCS optimizations

In [47]:
# Laser pulse parameters
theta = 160
beta = 90

# 1st laser
laser_params = []
laser = {
    'tau': 25e-15,
    'lam': 800e-9,
    'w0': 1*800e-9,
    'W': 25,
    'theta': 0,
    'phi': 0,
    'beta': 0,
    'solution': 'gaussian_paraxial'
}
laser['E0'] = float(W_to_E0(laser))
laser_params.append(laser)

# 2nd laser
laser = {
    'tau': 25e-15,
    'lam': 800e-9,
    'w0': 1*800e-9,
    'W': 25,
    'theta': theta,
    'phi': 0,
    'beta': beta,
    'solution': 'gaussian_paraxial'
}
laser['E0'] = float(W_to_E0(laser))
laser_params.append(laser)

laser_params = {f'laser_{i}': value for i,value in enumerate(laser_params)}

# Simulation box parameters
box_factors = {
    'long': 8,
    'trans': 18,
    't': 2
}

resolutions = {
    'spatial': 2,
    't': 1
}
simbox_params = {'box_factors': box_factors,
                 'resolutions': resolutions}

# Other parameters
mode = 'optuna'
geometry = 'z'
low_memory_mode = False
n_threads = 18
pol_idx = 0

# Optuna params
obj_param = 'N_disc_num'
n_trials = 6
consider_endpoints = True

data = {
    '_mode': mode,
    'lasers': laser_params,
    'simbox_params': simbox_params,
    'geometry': geometry,
    'low_memory_mode': low_memory_mode,
    'n_threads': n_threads,
    'pol_idx': pol_idx,
    'obj_param': obj_param,
    'n_trials': n_trials,
    'consider_endpoints': consider_endpoints
}

# Variable parameters 
param_key = 'w0'
grid = [1,9,800e-9,'uniform']
vary_params = {
    'laser_0':
        {param_key: grid}
}

# Save yaml files
folder = f'CCS_W'
subfolder = f'{param_key}_theta_{theta}_better_res'
save_path = f'{work_path}/runs/{mode}/{folder}/{subfolder}/'
Path(os.path.dirname(save_path)).mkdir(parents=True, exist_ok=True)

default_yaml = f'{save_path}default_params.yml'
write_yaml(default_yaml, data)

vary_yaml = f'{save_path}optuna_params.yml'
write_yaml(vary_yaml, vary_params)

In [48]:
jobname = f'm_opt_{param_key}'

script_path = f'{home_path}/cluster/optuna_cluster.py'
script_kwargs = f'--save_path={save_path}'
partition = 'long'

bash = template_optuna.format(jobname=f'{jobname}',
                              n_cpus=n_threads,
                              script_path=script_path,
                              script_kwargs=script_kwargs)

bash_path = f'{save_path}job.slurm'
with open(bash_path, 'w') as f:
    f.write(bash)

In [49]:
os.system(f'sbatch {bash_path}')

Submitted batch job 922310


0

#### ECS optimizations

In [6]:
# Laser pulse parameters
theta = 160
beta = 90

# 1st laser
laser_params = []
laser = {
    'tau': 25e-15,
    'lam': 800e-9,
    'w0x': 1*800e-9,
    'w0y': 1*800e-9,
    'W': 25,
    'theta': 0,
    'phi': 0,
    'beta': 0,
    'solution': 'gaussian_elliptical_paraxial'
}
laser['E0'] = float(W_to_E0(laser))
laser_params.append(laser)

# 2nd laser
laser = {
    'tau': 25e-15,
    'lam': 800e-9,
    # 'w0': 1*800e-9,
    'w0x': 1*800e-9,
    'w0y': 1*800e-9,
    'W': 25,
    'theta': theta,
    'phi': 0,
    'beta': beta,
    'solution': 'gaussian_elliptical_paraxial'
}
laser['E0'] = float(W_to_E0(laser))
laser_params.append(laser)

laser_params = {f'laser_{i}': value for i,value in enumerate(laser_params)}

# Simulation box parameters
box_factors = {
    'long': 7,
    'trans': 15,
    't': 2
}

resolutions = {
    'spatial': 1.5,
    't': 1
}
simbox_params = {'box_factors': box_factors,
                 'resolutions': resolutions}

# Other parameters
mode = 'optuna'
geometry = 'z'
low_memory_mode = False
n_threads = 24
pol_idx = 0

# Optuna params
obj_param = 'N_disc_num'
n_trials = 12
consider_endpoints = True
consider_prior = False
n_startup_trials = 6

data = {
    '_mode': mode,
    'lasers': laser_params,
    'simbox_params': simbox_params,
    'geometry': geometry,
    'low_memory_mode': low_memory_mode,
    'n_threads': n_threads,
    'pol_idx': pol_idx,
    'obj_param': obj_param,
    'n_trials': n_trials,
    'consider_endpoints': consider_endpoints,
    'consider_prior': consider_prior,
    'n_startup_trials': n_startup_trials,
}

# Variable parameters 
param_key = 'w0x_w0y'
# grid = [1,15,800e-9,'uniform']
# vary_params = {
#     'laser_0':
#         {param_key: grid}
# }
vary_params = {
    'laser_0':
        {'w0x': [1,9,800e-9,'uniform'],
         'w0y': [1,9,800e-9,'uniform'],},
}
# vary_params = {
#     'laser_0':
#         {'w0x': [1,9,800e-9,'uniform'],
#          'w0y': [1,9,800e-9,'uniform'],
#          'W': [0,50,1,'uniform']},
#     'laser_1':
#         {'w0x': [1,9,800e-9,'uniform'],
#          'w0y': [1,9,800e-9,'uniform']},
# }

# Save yaml files
folder = f'ECS_W'
subfolder = f'{param_key}_theta_{theta}_2_threads'
save_path = f'{work_path}/runs/{mode}/{folder}/{subfolder}/'
Path(os.path.dirname(save_path)).mkdir(parents=True, exist_ok=True)

default_yaml = f'{save_path}default_params.yml'
write_yaml(default_yaml, data)

vary_yaml = f'{save_path}optuna_params.yml'
write_yaml(vary_yaml, vary_params)

In [7]:
jobname = f'm_opt_{param_key}'

script_path = f'{home_path}/cluster/optuna_cluster.py'
script_kwargs = f'--save_path={save_path}'

bash = template_optuna.format(jobname=f'{jobname}',
                              n_cpus=n_threads,
                              script_path=script_path,
                              script_kwargs=script_kwargs)

bash_path = f'{save_path}job.slurm'
with open(bash_path, 'w') as f:
    f.write(bash)

In [8]:
os.system(f'sbatch {bash_path}')

Submitted batch job 1108854


0

#### Harmonics optimization

In [28]:
# Laser pulse parameters
theta = 180
beta = 90

lam0 = 800e-9
k0 = 2*np.pi/lam0

order = 0
gauss_type = 'gaussian_spectral'

w0_harm = 1
w0_usual = 1
# 1st laser
laser_params = []
for i in range(1,3):
    laser = {
        'tau': 20e-15,
        'lam': lam0/i,
        'w0': w0_harm*lam0/i,
        'W': 12.5,
        'theta': 0,
        'phi': 0,
        'beta': 0,
        'solution': gauss_type,
        'order': order,
    }
    laser['E0'] = float(W_to_E0(laser))
    laser_params.append(laser)

# 2nd laser
laser = {
    'tau': 20e-15,
    'lam': lam0,
    'w0': w0_usual*lam0,
    'W': 25,
    'theta': theta,
    'phi': 0,
    'beta': beta,
    'solution': gauss_type,
    'order': order,
}
laser['E0'] = float(W_to_E0(laser))
laser_params.append(laser)

laser_params = {f'laser_{i}': value for i,value in enumerate(laser_params)}

# Simulation box parameters
box_factors = {
    'long': 8,
    'trans': 20,
    't': 2
}

resolutions = {
    'spatial': 2,
    't': 1
}
simbox_params = {'box_factors': box_factors,
                 'resolutions': resolutions}

mode = 'optuna'
geometry = 'z'
low_memory_mode = False
n_threads = 20
pol_idx = 0

discernible_spectral = True
sphmap_params = {
    'order': 1,
    'angular_resolution': 0.5/180*np.pi
}

sph_limits = {
    'k': [1.5*k0, 2.5*k0],
    'theta': [0, np.pi]
}

# Optuna params
obj_param = 'N_disc_num'
n_trials = 10
consider_endpoints = True
consider_prior = False

data = {
    '_mode': mode,
    'lasers': laser_params,
    'simbox_params': simbox_params,
    'geometry': geometry,
    'low_memory_mode': low_memory_mode,
    'n_threads': n_threads,
    'pol_idx': pol_idx,
    'obj_param': obj_param,
    'n_trials': n_trials,
    'consider_endpoints': consider_endpoints,
    'discernible_spectral': discernible_spectral,
    'sphmap_params': sphmap_params,
    'sph_limits': sph_limits,
    'consider_prior': consider_prior,
}

# Variable parameters 
param_key = 'W'
grid = [0,50,1,'uniform']
vary_params = {
    'laser_0':
        {'W': grid}
}

# Save yaml files
folder = f'harmonics_W'
subfolder = f'{param_key}_2omega_channel'
save_path = f'{work_path}/runs/{mode}/{folder}/{subfolder}/'
Path(os.path.dirname(save_path)).mkdir(parents=True, exist_ok=True)

default_yaml = f'{save_path}default_params.yml'
write_yaml(default_yaml, data)

vary_yaml = f'{save_path}optuna_params.yml'
write_yaml(vary_yaml, vary_params)

In [29]:
jobname = f'm_opt_{param_key}'

script_path = f'{home_path}/cluster/optuna_cluster.py'
script_kwargs = f'--save_path={save_path}'

bash = template_optuna.format(jobname=f'{jobname}',
                              n_cpus=n_threads,
                              script_path=script_path,
                              script_kwargs=script_kwargs)

bash_path = f'{save_path}job.slurm'
with open(bash_path, 'w') as f:
    f.write(bash)

In [30]:
os.system(f'sbatch {bash_path}')

sbatch: error: invalid partition specified: hij-jena
sbatch: error: Batch job submission failed: Invalid partition name specified


256